# DerivKit: ForecastKit — 1D DALI Likelihood Demo

### Summary
This notebook demonstrates how to use **DerivKit's ForecastKit** to build a local
DALI (Derivative Approximation of the Likelihood, see arXiv:1401.6892) expansion of a 1D Gaussian
log-likelihood and compare it to:
- the **exact** log-likelihood, and
- the **quadratic Fisher approximation**.

We work with a simple one-parameter model
$ o(x) = 100\, e^{x^2} $, build the Fisher and DALI tensors at a fiducial
point $ x_0 $, and plot how well the local expansions track the exact
likelihood away from $ x_0 $.

### Usage
If you prefer to run the script version from the command line, you can use:

```bash
$ python demo-scripts/07-forecast-kit-dali.py --method adaptive
$ python demo-scripts/07-forecast-kit-dali.py --method adaptive --plot
```

In this notebook, just run the cells from top to bottom. You can change the
derivative backend (e.g. `"adaptive"`, `"finite"`) via the `method` variable
in the configuration cell below.

### What it does

- Defines a simple 1D model $ o(x) = 100\, e^{x^2} $.
- Sets a fiducial parameter value $ x_0 $ and a Gaussian data covariance
  with variance $ \sigma_o^2 $.
- Uses :class:`ForecastKit` to compute:
  - the Fisher matrix $ F $,
  - the DALI tensors $ G $ and $ H $ (doublet-DALI),
  at $ x_0 $.
- Evaluates on 1D grids:
  - the **exact** Gaussian log-likelihood,
  - the **Fisher** quadratic approximation,
  - the **Doublet-DALI** approximation including cubic and quartic terms.
- Plots all three on the same axes for visual comparison and prints a sanity
  check at $ x = x_0 $, where all approximations meet by construction.

### Notes

- In 1D, the tensors have shapes:
  - $ F \to (1, 1) $,
  - $ G \to (1, 1, 1) $,
  - $ H \to (1, 1, 1, 1) $.
- The code reads them as scalars `F[0,0]`, `G[0,0,0]`, `H[0,0,0,0]`.
- With $ x_0 \neq 0 $, the cubic term in DALI is generally non-zero,
  so the DALI curve is skewed relative to the Fisher Gaussian and tends to
  track the exact likelihood better away from $ x_0 $.

### Requirements

- `derivkit` installed and importable in your Python environment.
- (Optional) Your local DerivKit repo on `PYTHONPATH` if you are developing
  this notebook inside the repository.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from derivkit.forecast_kit import ForecastKit

# Optional: if you have DerivKit's plotting style helpers available, you can use them.
try:
    from utils.style import DEFAULT_COLORS, apply_plot_style
except ImportError:  # fallback if running outside the repo
    DEFAULT_COLORS = {
        "blue": "#1f77b4",
        "red": "#d62728",
        "yellow": "#ff7f0e",
    }

    def apply_plot_style():
        plt.style.use("default")
        plt.rcParams.update(
            {
                "figure.figsize": (8, 5),
                "axes.grid": False,
                "font.size": 14,
            }
        )

print("Imports complete.")

In [ ]:
def test_model_1d(param_list) -> np.ndarray:
    """Returns one observable with a quadratic dependence on one parameter.

    Model:
        o(x) = 100 * exp(x^2)

    Args:
        param_list: list of one parameter [x].

    Returns:
        np.ndarray
            One-element array [o].
    """
    x = float(param_list[0])
    obs = 1e2 * np.exp(x**2)
    return np.array([obs], dtype=float)


def loglike_1d_exact(sigma_o: float, fiducial_x: float, x: float) -> float:
    """Exact Gaussian log-likelihood for one observable with variance sigma_o^2.

    log L(x) = -0.5 * ((o(x) - o(x0)) / sigma_o)^2

    Args:
        sigma_o:
            Standard deviation of the observable.
        fiducial_x:
            Fiducial parameter value x0.
        x:
            Parameter value at which to evaluate the log-likelihood.

    Returns:
        float:
            Exact log-likelihood value.
    """
    delta_o = test_model_1d([x]) - test_model_1d([fiducial_x])
    return float(-0.5 * (delta_o[0] / sigma_o) ** 2)


def loglike_1d_approx(tensors: list[np.ndarray], fiducial_x: float, x: float) -> float:
    """Approximate log-likelihood using Fisher + doublet-DALI terms.

    In 1D, the expansion reads:
        log L(x) ≈ -1/2 F Δx^2 - 1/2 G Δx^3 - 1/8 H Δx^4,
    where Δx = x - x0.

    Args:
        tensors:
            List of tensors [F, G, H] where
            F is (1,1), G is (1,1,1), H is (1,1,1,1) in 1D.
        fiducial_x:
            Fiducial parameter value x0.
        x:
            Parameter value at which to evaluate the log-likelihood.

    Returns:
        float:
            Approximate log-likelihood value.
    """
    dx = x - fiducial_x
    loglike = 0.0

    if len(tensors) >= 1:
        f = np.asarray(tensors[0])
        loglike += -0.5 * float(f[0, 0]) * dx**2

    if len(tensors) >= 3:
        g = np.asarray(tensors[1])
        h = np.asarray(tensors[2])
        loglike += -0.5 * float(g[0, 0, 0]) * dx**3
        loglike += -0.125 * float(h[0, 0, 0, 0]) * dx**4

    return float(loglike)


print("Model and likelihood helpers defined.")

In [ ]:
# Configuration

fiducial_values = [0.1]  # x0
covmat = np.array([[1.0]], dtype=float)  # C = [[sigma_o^2]] with sigma_o = 1

fiducial_x = float(fiducial_values[0])  # our assumed "true" parameter value
sigma_o = float(np.sqrt(covmat[0, 0]))  # standard deviation of the observable

# Derivative backend for ForecastKit ("adaptive", "finite", etc.)
method = "adaptive"

# Grids
xgrid = np.linspace(-1.0, 1.0, 1000)
xgrid_sparse = np.linspace(-0.2, 0.2, 100)

print(f"fiducial_x = {fiducial_x}")
print(f"sigma_o = {sigma_o}")
print(f"method = '{method}'")

In [ ]:
# Build ForecastKit object and compute tensors

observables = test_model_1d

fk = ForecastKit(function=observables,
                 theta0=np.array(fiducial_values, dtype=float),
                 cov=covmat)

fisher_matrix = fk.fisher(method=method)  # shape (1, 1)
dali_dict = fk.dali(method=method)  
dali_g = dali_dict[2][0]  # shape (1,1,1)
dali_h = dali_dict[2][1]  # shape (1,1,1,1)

print("Fisher matrix F:\n", fisher_matrix)
print("\nDALI G tensor shape:", dali_g.shape)
print("DALI H tensor shape:", dali_h.shape)

In [ ]:
# Evaluate exact and approximate log-likelihoods on grids

tensors = [fisher_matrix, dali_g, dali_h]

exact_like = np.array([loglike_1d_exact(sigma_o, fiducial_x, x) for x in xgrid])
fisher_like = np.array([loglike_1d_approx([fisher_matrix], fiducial_x, x) for x in xgrid])
dali_like_sparse = np.array([loglike_1d_approx(tensors, fiducial_x, x) for x in xgrid_sparse])

print("Computed exact, Fisher, and DALI log-likelihoods on the grids.")

In [ ]:
# Plot comparison: exact vs Fisher vs Doublet-DALI in 1D

apply_plot_style()
fig, ax = plt.subplots()

blue = DEFAULT_COLORS.get("blue", "#1f77b4")
red = DEFAULT_COLORS.get("red", "#d62728")
yellow = DEFAULT_COLORS.get("yellow", "#ff7f0e")

ax.plot(xgrid, exact_like, label="Exact Likelihood", linewidth=3, color=red)
ax.plot(xgrid, fisher_like, label="Fisher Matrix", linewidth=3, linestyle="-", color=yellow)
ax.plot(xgrid_sparse, dali_like_sparse, label="Doublet DALI",
        markersize=6, color=blue, linestyle="--", linewidth=3)

ax.set_title(r"$\mathrm{observable} = 100 \cdot e^{x^2}$", fontsize=20)
ax.set_xlabel(r"$x$", fontsize=20)
ax.set_ylabel(r"$\log P$", fontsize=20)
ax.set_xlim(-0.3, 0.6)
ax.set_ylim(-0.65, 0.05)
ax.legend(fontsize=14, framealpha=1.0)
ax.minorticks_off()

plt.tight_layout()
plt.show()

## 2D DALI vs Fisher contours (bonus)

As a bonus, this cell builds a simple **two-parameter** model and compares
the closed $1\sigma/2\sigma$contours of

- the **exact** Gaussian log-likelihood,
- the **Fisher quadratic** approximation, and
- the **Doublet-DALI** approximation,

using Matplotlib's `contour` on a common $(\theta_1, \theta_2)\) grid.

This is conceptually the same as the 1D demo above, but now in 2D so that
we can draw proper contour lines in parameter space.


In [ ]:
# 2D example: DALI vs Fisher closed contours

from matplotlib.lines import Line2D

# Toggles
SHOW_1SIGMA = True  # Δχ² = 2.30 (2D)
SHOW_2SIGMA = True  # Δχ² = 6.17 (2D)
SHOW_EXACT = True  # also draw the exact likelihood contours
GRID_N = 300  # grid resolution per axis
BOX_SIGMA = 4.0  # how many Fisher-σ to include along each parameter axis


def model_2d(theta_list):
    """Simple 2D toy model with two observables.

    Args:
        theta_list : array-like of length 2
            [θ1, θ2] ≡ [x, ε].

    Returns:
        np.ndarray, shape (2,)
            Two observables:
              o1 = (1 + ε)      * 100 * exp(x^2)
              o2 = (1 + 0.3 ε)  *  40 * exp(0.5 x)
    """
    x, eps = float(theta_list[0]), float(theta_list[1])
    o1 = (1.0 + eps) * (1e2 * np.exp(x**2))
    o2 = (1.0 + 0.3 * eps) * (4e1 * np.exp(0.5 * x))
    return np.array([o1, o2])


def loglike_exact_2d(cov: np.ndarray, theta0: np.ndarray, theta: np.ndarray) -> float:
    """Exact Gaussian log-likelihood in 2D parameter space.

    Args:
        cov : np.ndarray, shape (2, 2)
            Data covariance matrix.
        theta0 : np.ndarray, shape (2,)
            Fiducial parameter values [θ1_0, θ2_0].
        theta : np.ndarray, shape (2,)
            Parameter values [θ1, θ2] at which to evaluate the log-likelihood.

    Returns:
        Exact log-likelihood value.
    """
    o = model_2d(theta)
    o0 = model_2d(theta0)
    r = o - o0
    Ci = np.linalg.pinv(cov)
    return -0.5 * float(r @ (Ci @ r))


def loglike_fisher_2d(F: np.ndarray, dtheta: np.ndarray) -> float:
    """Fisher quadratic approximation in 2D."""
    return -0.5 * float(dtheta @ F @ dtheta)


def loglike_dali_2d(F: np.ndarray, G: np.ndarray, H: np.ndarray, dtheta: np.ndarray) -> float:
    """Doublet-DALI approximation in 2D.

    log L(θ) ≈ -1/2 F_{ij} Δθ_i Δθ_j
               -1/2 G_{ijk} Δθ_i Δθ_j Δθ_k
               -1/8 H_{ijkl} Δθ_i Δθ_j Δθ_k Δθ_l
    """
    q = -0.5 * float(dtheta @ F @ dtheta)
    cubic = -0.5 * float(np.einsum("ijk,i,j,k", G, dtheta, dtheta, dtheta))
    quart = -0.125 * float(np.einsum("ijkl,i,j,k,l", H, dtheta, dtheta, dtheta, dtheta))
    return q + cubic + quart


# Build tensors at a 2D fiducial

theta0_2d = np.array([0.10, 0.0])  # [θ1, θ2]
covmat_2d = np.diag([1.0, 1.0])  # 2 observables, unit covariance

fk2d = ForecastKit(model_2d, theta0_2d, covmat_2d)
F2 = fk2d.fisher(method=method)
dali_dict = fk2d.dali(method=method)
G2 = dali_dict[2][0]
H2 = dali_dict[2][1]

print("2D Fisher matrix F:\n", F2)
print("\n2D DALI G tensor shape:", G2.shape)
print("2D DALI H tensor shape:", H2.shape)

# Exact max value at the fiducial and Δχ² levels
L0_2d = loglike_exact_2d(covmat_2d, theta0_2d, theta0_2d)
dchis = []
if SHOW_1SIGMA:
    dchis.append(2.30)  # 1σ in 2D
if SHOW_2SIGMA:
    dchis.append(6.17)  # 2σ in 2D

# Convert Δχ² → log L levels (must be increasing for contour)
L_levels_2d = np.array([L0_2d - 0.5 * d for d in dchis], dtype=float)
L_levels_2d = np.unique(L_levels_2d)
L_levels_2d.sort()

# Build a rectangular grid aligned with Fisher covariance axes
cov_theta = np.linalg.pinv(F2)
sigma_theta = np.sqrt(np.clip(np.diag(cov_theta), 0.0, None))

t1 = np.linspace(theta0_2d[0] - BOX_SIGMA * sigma_theta[0],
                 theta0_2d[0] + BOX_SIGMA * sigma_theta[0],
                 GRID_N)
t2 = np.linspace(theta0_2d[1] - BOX_SIGMA * sigma_theta[1],
                 theta0_2d[1] + BOX_SIGMA * sigma_theta[1],
                 GRID_N)
T1, T2 = np.meshgrid(t1, t2, indexing="xy")

TH = np.stack([T1.ravel(), T2.ravel()], axis=1)
dTH = TH - theta0_2d[None, :]

# Evaluate fields
Z_exact_2d = np.array(
    [loglike_exact_2d(covmat_2d, theta0_2d, th) for th in TH],
    dtype=float,
).reshape(T1.shape)

Z_dali_2d = np.array(
    [loglike_dali_2d(F2, G2, H2, dt) for dt in dTH],
    dtype=float,
).reshape(T1.shape)

Z_fish_2d = np.array(
    [loglike_fisher_2d(F2, dt) for dt in dTH],
    dtype=float,
).reshape(T1.shape)


In [ ]:
# Contour plot

apply_plot_style()
fig, ax = plt.subplots(figsize=(6, 6))

blue = DEFAULT_COLORS.get("blue", "#1f77b4")
yellow = DEFAULT_COLORS.get("yellow", "#F2C94C")
red = DEFAULT_COLORS.get("red", "#d62728")
lw = 2.0

# Fisher (yellow)
cs_fish = ax.contour(
    T1, T2, Z_fish_2d, levels=L_levels_2d,
    colors=[yellow] * len(L_levels_2d),
    linestyles="-",
    linewidths=lw,
)

# DALI (blue)
cs_dali = ax.contour(
    T1, T2, Z_dali_2d, levels=L_levels_2d,
    colors=[blue] * len(L_levels_2d),
    linestyles="-",
    linewidths=lw,
)

# Exact (red, dashed) if requested
if SHOW_EXACT:
    cs_exact = ax.contour(
        T1, T2, Z_exact_2d, levels=L_levels_2d,
        colors=[red] * len(L_levels_2d),
        linestyles="--",
        linewidths=lw,
    )

# Fiducial point
ax.scatter([theta0_2d[0]], [theta0_2d[1]], marker="x", s=80, color=red, zorder=10)

# Legend
handles = [
    Line2D([0], [0], color=blue, lw=lw, ls="-", label="Doublet DALI"),
    Line2D([0], [0], color=yellow, lw=lw, ls="-", label="Fisher"),
]
if SHOW_EXACT:
    handles.insert(0, Line2D([0], [0], color=red, lw=lw, ls="--", label="Exact"))
handles.append(Line2D([0], [0], color=red, lw=0, marker="x", ms=8, label="Fiducial"))

ax.legend(handles=handles, fontsize=12, framealpha=1.0, loc="best")

ax.set_xlabel(r"parameter $\theta_1$", fontsize=16)
ax.set_ylabel(r"parameter $\theta_2$", fontsize=16)
ax.tick_params(labelsize=12)
ax.minorticks_off()

plt.tight_layout()
plt.show()